In [ ]:
import strax 
import straxen 
import numpy as np
import matplotlib.pyplot as plt
import cutax
import pandas as pd

In [ ]:
1+1

In [ ]:
strax.__version__

In [ ]:
cutax.__version__

In [ ]:
straxen.__version__

In [ ]:
st = cutax.contexts.xenonnt_v8()

In [ ]:
st = straxen.contexts.xenonnt_simulation()

In [ ]:
pb_data = np.load('./saved_data/kr_clean_pb_data_v2.npz')['arr_0']

In [ ]:
st.select_runs(available=('event_info'),run_mode='tpc_ar37',
                             include_tags="_sr0",
                             exclude_tags=('messy','bad','nonsr0_configuration','ramp down','ramp_down','ramp_up','ramp up','hot_spot','abandon','warm_spot'))

In [ ]:
cal_run_id = '034373'

In [ ]:
kr_event_info = st.get_array(run_id = cal_run_id , targets='event_info')

In [ ]:
kr_events_info_cuts = st.get_array(cal_run_id, targets =('event_info', 'cut_fiducial_volume', 'cut_s2_width', 'cut_s2_single_scatter',
                                                        'cut_s2_pattern', 'cut_s2_recon_pos_diff', 'cut_cs2_area_fraction_top'))

In [ ]:
cut_list = ['cut_fiducial_volume', 'cut_s2_width', 'cut_s2_single_scatter',
            'cut_s2_pattern', 'cut_s2_recon_pos_diff', 'cut_cs2_area_fraction_top']

In [ ]:
cut_list[0]

In [ ]:
len(kr_events_info_cuts['s1_index'])

In [ ]:
def apply_cuts_to_event_info(event_info, cut_list):
    event_info_cuts = event_info
    for i in np.arange(len(cut_list)):
        event_info_cuts = event_info_cuts[event_info_cuts[cut_list[i]] == True]
        
    return event_info_cuts

In [ ]:
def compute_s1_boundary(parm, area):
    boundary_line = parm[0]*np.exp(-area/parm[1]) + parm[2]
    
    return boundary_line

In [ ]:
events_with_cuts = apply_cuts_to_event_info(kr_events_info_cuts, cut_list)

In [ ]:
len(events_with_cuts)

In [ ]:
pb_data_SOM2cls = np.load('./saved_data/kr_034373_pb_SOM2cls.npz')['arr_0']

In [ ]:
def match_event_basics_to_data(event_basics, peak_info):
    """
    Feed in event basics and peaks or peaks basics and return the information in peak basics regarding the event.
    Note some data will be repreated as every event has multiple peaks.
    """
    # need empty array that I can just put things in
    # what if a particular value is never found
    eb_info = np.array([])  #event basics index
    peak_basics_index = np.array([])
    nan_array = np.repeat(float("nan"), 1)
    
    for jj in np.arange(len(peak_info)):
        
        if any(peak_info['time'][jj] == event_basics['s1_time']): #if the value is an S1
            index = int(np.argwhere(peak_info['time'][jj] == event_basics['s1_time'])) #index in peak basics
            eb_info = np.append(eb_info, index )
            peak_basics_index = np.append(peak_basics_index, jj)

        elif any(peak_info['time'][jj] == event_basics['s2_time']): #if the value is an S1
            index = int(np.argwhere(peak_info['time'][jj] == event_basics['s2_time']))
            eb_info = np.append(eb_info, index )
            peak_basics_index = np.append(peak_basics_index, jj)
            
    eb_index = eb_info
            
    return eb_index, peak_basics_index

In [ ]:
eb_index, pb_index = match_event_basics_to_data(events_with_cuts, pb_data_SOM2cls)

In [ ]:
data_pb_cuts_2clsSOM = pb_data_SOM2cls[pb_index.astype(int)]

In [ ]:
fig = plt.figure(figsize=(10,7))

plt.scatter(pb_data_SOM2cls['area'][pb_data_SOM2cls['type'] == 1], 
            pb_data_SOM2cls['rise_time'][pb_data_SOM2cls['type'] == 1], 
            s=0.5, alpha = 0.5, color = 'blue')
plt.scatter(pb_data_SOM2cls['area'][pb_data_SOM2cls['type'] == 2], 
            pb_data_SOM2cls['rise_time'][pb_data_SOM2cls['type'] == 2], 
            s=0.5, alpha = 0.5, color = 'green')
#plt.scatter(kr_pb_array['area'][kr_pb_array['type'] == 1], kr_pb_array['rise_time'][kr_pb_array['type'] == 1], s=0.1, color = 'blue')
plt.xscale('log')
plt.yscale('log')
plt.xlabel('Area [PE]')
plt.ylabel('Rise Time [ns]')
plt.xlim(1,100)
plt.ylim(1,300)

In [ ]:
fig = plt.figure(figsize=(10,7))

plt.scatter(pb_data_SOM2cls['area'][pb_data_SOM2cls['type'] == 1], 
            pb_data_SOM2cls['rise_time'][pb_data_SOM2cls['type'] == 1], 
            s=0.5, alpha = 0.5, color = 'blue')
plt.scatter(pb_data_SOM2cls['area'][pb_data_SOM2cls['type'] == 2], 
            pb_data_SOM2cls['rise_time'][pb_data_SOM2cls['type'] == 2], 
            s=0.5, alpha = 0.5, color = 'green')
#plt.scatter(kr_pb_array['area'][kr_pb_array['type'] == 1], kr_pb_array['rise_time'][kr_pb_array['type'] == 1], s=0.1, color = 'blue')
plt.xscale('log')
plt.yscale('log')
plt.xlabel('Area [PE]')
plt.ylabel('Rise Time [ns]')
plt.xlim(1,50)
plt.ylim(10,500)

In [ ]:
parm = np.array([50, 80, 12])

In [ ]:
parm2 = np.array([100, 80, 12])

In [ ]:
parm3 = np.array([50, 80, 60])

In [ ]:
st_s1_data = pb_data[pb_data['type'] == 1]
st_s2_data = pb_data[pb_data['type'] == 2]

In [ ]:
area_linspace =  np.linspace(1, 100, 200)

In [ ]:
fig = plt.figure(figsize=(6,5))

plt.scatter(pb_data['area'][pb_data['type'] == 1], 
            pb_data['rise_time'][pb_data['type'] == 1], 
            s=0.5, alpha = 0.5, color = 'blue', label = 'S1')
plt.scatter(pb_data['area'][pb_data['type'] == 2], 
            pb_data['rise_time'][pb_data['type'] == 2], 
            s=0.5, alpha = 0.5, color = 'green', label = 'S2')
plt.plot(area_linspace, compute_s1_boundary(parm2, area_linspace),
         '--' ,color = 'black', label = 'Straxen Peaklet Boundary')
#plt.scatter(kr_pb_array['area'][kr_pb_array['type'] == 1], kr_pb_array['rise_time'][kr_pb_array['type'] == 1], s=0.1, color = 'blue')
plt.xscale('log')
plt.yscale('log')
plt.xlabel('Area [PE]')
plt.ylabel('Rise Time [ns]')
plt.legend(markerscale=8)
plt.xlim(1,50)
plt.ylim(10,500)
plt.savefig('./figures/Straxen_cls_risetime_v_area_boundary.png')

In [ ]:
fig = plt.figure(figsize=(6,5))

plt.scatter(pb_data['area'][pb_data['type'] == 1], 
            pb_data['rise_time'][pb_data['type'] == 1], 
            s=0.5, alpha = 0.5, color = 'blue', label = 'S1')
plt.scatter(pb_data['area'][pb_data['type'] == 2], 
            pb_data['rise_time'][pb_data['type'] == 2], 
            s=0.5, alpha = 0.5, color = 'green', label = 'S2')
plt.plot(area_linspace, compute_s1_boundary(parm2, area_linspace),
         '--' ,color = 'black', label = 'Straxen Peaklet Boundary')
#plt.scatter(kr_pb_array['area'][kr_pb_array['type'] == 1], kr_pb_array['rise_time'][kr_pb_array['type'] == 1], s=0.1, color = 'blue')
plt.xscale('log')
plt.yscale('log')
plt.xlabel('Area [PE]')
plt.ylabel('Rise Time [ns]')
plt.legend(markerscale=8)
plt.xlim(1,50)
plt.ylim(10,500)
plt.savefig('./figures/Straxen_cls_risetime_v_area_boundary.png', dpi=1200)

In [ ]:
fig = plt.figure(figsize=(6,5))

parm2 = np.array([50, 12, 80])
plt.scatter(pb_data['area'][pb_data['type'] == 1], 
            pb_data['rise_time'][pb_data['type'] == 1], 
            s=0.5, alpha = 0.5, color = 'blue', label = 'Straxen S1')
plt.scatter(pb_data['area'][pb_data['type'] == 2], 
            pb_data['rise_time'][pb_data['type'] == 2], 
            s=0.5, alpha = 0.5, color = 'green', label = 'Straxen S2')
plt.plot(area_linspace, compute_s1_boundary(parm2, area_linspace),
         '--' ,color = 'black', label = 'Straxen Peaklet Boundary')
#plt.scatter(kr_pb_array['area'][kr_pb_array['type'] == 1], kr_pb_array['rise_time'][kr_pb_array['type'] == 1], s=0.1, color = 'blue')
#plt.xscale('log')
#plt.yscale('log')
plt.xlabel('Area [PE]')
plt.ylabel('Rise Time [ns]')
plt.legend(markerscale=8, loc = 2)
plt.xlim(1,50)
plt.ylim(10,300)
plt.savefig('./figures/Straxen_cls_risetime_v_area_boundary.png', dpi=1200)

In [ ]:
pb_s1_data = pb_data[pb_data['type'] == 1]
pb_s1_data_high_aft = pb_s1_data[pb_s1_data['area_fraction_top'] > 0.7]

pb_s2_data = pb_data[pb_data['type'] == 2]
pb_s2_data_low_aft = pb_s2_data[pb_s2_data['area_fraction_top'] < 0.3]

In [ ]:
fig = plt.figure(figsize=(7,5))

parm2 = np.array([50, 12, 80])
vmin = 0
vmax = 1
plt.scatter(pb_data['area'][pb_data['type'] == 1], 
            pb_data['rise_time'][pb_data['type'] == 1], 
            c = pb_data['area_fraction_top'][pb_data['type'] == 1],
            s=5, alpha = 0.5, marker = '^',
            vmin=vmin, vmax=vmax, label = 'Straxen S1',
            cmap=cm.get_cmap('viridis'))
plt.scatter(pb_data['area'][pb_data['type'] == 2], 
            pb_data['rise_time'][pb_data['type'] == 2], 
            c = pb_data['area_fraction_top'][pb_data['type'] == 2],
            s=5, alpha = 0.5, vmin=vmin, vmax=vmax, 
            label = 'Straxen S2', marker = 'v',
            cmap=cm.get_cmap('viridis'))
plt.scatter(pb_s1_data_high_aft['area'],
            pb_s1_data_high_aft['rise_time'], s = 10,
            marker = 'x', color = 'red', label = 'high aft Straxen S1')
plt.scatter(pb_s2_data_low_aft['area'],
            pb_s2_data_low_aft['rise_time'], s = 10,
            marker = 'x', color = 'black', label = 'low aft Straxen S2')
plt.plot(area_linspace, compute_s1_boundary(parm2, area_linspace),
         '--' ,color = 'black', label = 'Straxen Peaklet Boundary')
plt.colorbar(label = 'AFT')
#plt.scatter(kr_pb_array['area'][kr_pb_array['type'] == 1], kr_pb_array['rise_time'][kr_pb_array['type'] == 1], s=0.1, color = 'blue')
#plt.xscale('log')
#plt.yscale('log')
plt.xlabel('Area [PE]')
plt.ylabel('Rise Time [ns]')
plt.legend(markerscale=3, loc = 1)
plt.xlim(1,50)
plt.ylim(70,125)
#plt.savefig('./figures/Straxen_cls_risetime_v_area_boundary.png', dpi=1200)

In [ ]:
pb_s1_data_st = pb_data[pb_data['type'] == 1]
pb_s1_data_high_aft = pb_s1_data_st[pb_s1_data_st['area_fraction_top'] > 0.8]

pb_s2_data_st = pb_data[pb_data['type'] == 2]
pb_s2_data_low_aft = pb_s2_data_st[pb_s2_data_st['area_fraction_top'] < 0.2]

In [ ]:
len(pb_s1_data_high_aft_som)

In [ ]:
len(pb_s1_data_high_aft_st)

In [ ]:
len(pb_s2_data_low_aft_st)

In [ ]:
len(pb_s2_data_low_aft_som)

In [ ]:
fig = plt.figure(figsize=(7,5))

parm2 = np.array([50, 12, 80])
vmin = 0
vmax = 1
plt.scatter(pb_data['area'][pb_data['type'] == 1], 
            pb_data['rise_time'][pb_data['type'] == 1], 
            c = pb_data['area_fraction_top'][pb_data['type'] == 1],
            s=10, alpha = 0.5, marker = '^',
            vmin=vmin, vmax=vmax, label = 'Straxen S1',
            cmap=cm.get_cmap('viridis'))
plt.scatter(pb_data['area'][pb_data['type'] == 2], 
            pb_data['rise_time'][pb_data['type'] == 2], 
            c = pb_data['area_fraction_top'][pb_data['type'] == 2],
            s=10, alpha = 0.5, vmin=vmin, vmax=vmax, 
            label = 'Straxen S2', marker = 'v',
            cmap=cm.get_cmap('viridis'))
plt.scatter(pb_s1_data_high_aft_st['area'],
            pb_s1_data_high_aft_st['rise_time'], s = 10,
            marker = 'x', color = 'lime', label = 'aft > 0.8, Straxen S1')
plt.scatter(pb_s2_data_low_aft_st['area'],
            pb_s2_data_low_aft_st['rise_time'], s = 10,
            marker = 'x', color = 'red', label = 'aft < 0.2, Straxen S2')
plt.plot(area_linspace, compute_s1_boundary(parm2, area_linspace),
         '--' ,color = 'black', label = 'Straxen Peaklet Boundary')
plt.colorbar(label = 'AFT')
#plt.scatter(kr_pb_array['area'][kr_pb_array['type'] == 1], kr_pb_array['rise_time'][kr_pb_array['type'] == 1], s=0.1, color = 'blue')
#plt.xscale('log')
#plt.yscale('log')
plt.xlabel('Area [PE]')
plt.ylabel('Rise Time [ns]')
plt.legend(markerscale=3, loc = 1)
plt.xlim(1,50)
plt.ylim(70,125)
#plt.savefig('./figures/Risetime_v_area_straxen_error.png', dpi = 1200)

In [ ]:
fig = plt.figure(figsize=(7,5))

parm2 = np.array([50, 12, 80])
vmin = 0
vmax = 1

plt.scatter(pb_s1_data_high_aft_st['area'],
            pb_s1_data_high_aft_st['rise_time'], s = 10,
            marker = 'x', color = 'cornflowerblue', label = 'aft > 0.8, Straxen S1')
plt.scatter(pb_s2_data_low_aft_st['area'],
            pb_s2_data_low_aft_st['rise_time'], s = 10,
            marker = 'x', color = 'magenta', label = 'aft < 0.2, Straxen S2')
plt.plot(area_linspace, compute_s1_boundary(parm2, area_linspace),
         '--' ,color = 'black', label = 'Straxen Peaklet Boundary')
#plt.colorbar(label = 'AFT')
#plt.scatter(kr_pb_array['area'][kr_pb_array['type'] == 1], kr_pb_array['rise_time'][kr_pb_array['type'] == 1], s=0.1, color = 'blue')
#plt.xscale('log')
#plt.yscale('log')
plt.xlabel('Area [PE]')
plt.ylabel('Rise Time [ns]')
plt.legend(markerscale=3, loc = 1)
plt.xlim(1,50)
plt.ylim(70,125)

In [ ]:
fig = plt.figure(figsize=(6,5))

plt.scatter(pb_data['area'][pb_data['type'] == 1], 
            pb_data['rise_time'][pb_data['type'] == 1], 
            s=1, alpha = 0.5, color = 'blue', label = 'S1')
plt.scatter(pb_data['area'][pb_data['type'] == 2], 
            pb_data['rise_time'][pb_data['type'] == 2], 
            s=1, alpha = 0.5, color = 'green', label = 'S2')
plt.plot(area_linspace, compute_s1_boundary(parm2, area_linspace),
         '--' ,color = 'black', label = "Straxen Peaklet Boundary")
#plt.scatter(kr_pb_array['area'][kr_pb_array['type'] == 1], kr_pb_array['rise_time'][kr_pb_array['type'] == 1], s=0.1, color = 'blue')
plt.xscale('log')
plt.yscale('log')
plt.xlabel('Area [PE]')
plt.ylabel('Rise Time [ns]')
plt.xlim(1,50)
plt.ylim(10,500)
plt.legend(markerscale=8)
#plt.title("")

In [ ]:
fig = plt.figure(figsize=(6,5))

plt.scatter(pb_data_s1['area'], 
            pb_data_s1['rise_time'], 
            s=1, alpha = 0.5, color = 'blue', label = 'S1')
plt.scatter(pb_data_s2['area'], 
            pb_data_s2['rise_time'], 
            s=1, alpha = 0.5, color = 'green', label = 'S2')
plt.plot(area_linspace, compute_s1_boundary(parm2, area_linspace),
         '--' ,color = 'black', label = "Straxen Peaklet Boundary")
#plt.scatter(kr_pb_array['area'][kr_pb_array['type'] == 1], kr_pb_array['rise_time'][kr_pb_array['type'] == 1], s=0.1, color = 'blue')
plt.xscale('log')
plt.yscale('log')
plt.xlabel('Area [PE]')
plt.ylabel('Rise Time [ns]')
plt.xlim(1,50)
plt.ylim(10,500)
plt.legend(markerscale=8)
#plt.title("")

In [ ]:
aft_bound_parm = np.array([-1, 2.6, 0.6, 100])

In [ ]:
aft_line = np.linspace(0,1,100)

In [ ]:
def upper_rise_time_aft_boundary(aft, slope, offset, aft_boundary, flat_threshold):
        """
        Function which computes the upper rise time boundary as a function
        of area fraction top.
        """
        res = 10**(slope * aft + offset)
        res[aft >= aft_boundary] = flat_threshold
        return res

In [ ]:
fig = plt.figure(figsize=(6,5))

plt.scatter(st_s1_data['area_fraction_top'], 
            st_s1_data['rise_time'], 
            s=0.5, alpha = 0.5, color = 'blue', label = 'S1')
plt.scatter(st_s2_data['area_fraction_top'], 
            st_s2_data['rise_time'], 
            s=0.5, alpha = 0.5, color = 'green', label = 'S2')
plt.plot(aft_line, upper_rise_time_aft_boundary(aft_line, aft_bound_parm[0],
                                               aft_bound_parm[1],aft_bound_parm[2],
                                               aft_bound_parm[3]),'--' ,color = 'black', label = "Straxen AFT boundary")
#plt.scatter(kr_pb_array['area'][kr_pb_array['type'] == 1], kr_pb_array['rise_time'][kr_pb_array['type'] == 1], s=0.1, color = 'blue')
#plt.xscale('log')
plt.yscale('log')
plt.xlabel('Area Fraction Top')
plt.ylabel('Rise Time [ns]')
plt.xlim(0,1)
plt.ylim(10,1000)
plt.legend(markerscale=8)
plt.savefig('./figures/Straxen_cls_risetime_v_AFT_boundary.png', dpi = 1200)

In [ ]:
fig = plt.figure(figsize=(10,7))

plt.scatter(pb_data['area'][pb_data['type'] == 1], 
            pb_data['rise_time'][pb_data['type'] == 1], 
            s=0.5, alpha = 0.5, color = 'blue')
plt.scatter(pb_data['area'][pb_data['type'] == 2], 
            pb_data['rise_time'][pb_data['type'] == 2], 
            s=0.5, alpha = 0.5, color = 'green')
plt.plot(area_linspace, compute_s1_boundary(parm2, area_linspace),'--' ,color = 'black')
#plt.scatter(kr_pb_array['area'][kr_pb_array['type'] == 1], kr_pb_array['rise_time'][kr_pb_array['type'] == 1], s=0.1, color = 'blue')
plt.xscale('log')
plt.yscale('log')
plt.xlabel('Area [PE]')
plt.ylabel('Rise Time [ns]')
plt.xlim(1,50)
plt.ylim(10,500)

In [ ]:
pb_data_s2 = pb_data_SOM2cls[pb_data_SOM2cls['type'] == 2]
pb_data_s1 = pb_data_SOM2cls[pb_data_SOM2cls['type'] == 1]

In [ ]:
from matplotlib import cm

In [ ]:
from matplotlib.lines import Line2D

In [ ]:
fig = plt.figure(figsize=(8,5))

vmin = 0
vmax = 1
m1 = plt.scatter(pb_data_s2['area'][pb_data_s2['area'] < 70], 
            pb_data_s2['rise_time'][pb_data_s2['area'] < 70], 
            s=5, alpha = 0.5, c= pb_data_s2['area_fraction_top'][pb_data_s2['area'] < 70],
            vmin=vmin, vmax=vmax, label = 'SOM S2', 
            cmap=cm.get_cmap('Reds'))
plt.colorbar(m1, label = 'S2 AFT')
m2 = plt.scatter(pb_data_s1['area'][pb_data_s1['area'] < 50], 
            pb_data_s1['rise_time'][pb_data_s1['area'] < 50], 
            s=5, alpha = 0.5, c = pb_data_s1['area_fraction_top'][pb_data_s1['area'] < 50],
            vmin=vmin, vmax=vmax, label = 'SOM S1',
            cmap=cm.get_cmap('Greens'))
plt.colorbar(m2, label = 'S1 AFT')
plt.plot(area_linspace, compute_s1_boundary(parm2, area_linspace),
         '--' ,color = 'black', label = 'Straxen Boundary')
#plt.scatter(kr_pb_array['area'][kr_pb_array['type'] == 1], kr_pb_array['rise_time'][kr_pb_array['type'] == 1], s=0.1, color = 'blue')
plt.xscale('log')
plt.yscale('log')
plt.xlabel('Area [PE]')
plt.ylabel('Rise Time [ns]')
legend_elements = [Line2D([0], [0], marker='o', color='green', label='SOM S1',markerfacecolor='green',linewidth=0, markersize=5),
                   Line2D([0], [0], marker='o', color='red', label='SOM S2',markerfacecolor='red',linewidth=0, markersize=5),
                   Line2D([0], [0], linestyle='--', color='black', label='Straxen Boundary',markerfacecolor='black',linewidth=2, markersize=5)]

plt.legend(handles=legend_elements, loc=0,prop={'size':12})
#plt.colorbar()
#plt.legend(markerscale=4)
plt.xlim(1,50)
plt.ylim(10,500)

In [ ]:
fig = plt.figure(figsize=(8,5))

vmin = 0
vmax = 1
m1 = plt.scatter(pb_data_s2['area'][pb_data_s2['area'] < 70], 
            pb_data_s2['rise_time'][pb_data_s2['area'] < 70], 
            s=5, alpha = 0.5, c= pb_data_s2['area_fraction_top'][pb_data_s2['area'] < 70],
            vmin=vmin, vmax=vmax, label = 'SOM S2', 
            cmap=cm.get_cmap('Reds'))
plt.colorbar(m1, label = 'S2 AFT')
m2 = plt.scatter(pb_data_s1['area'][pb_data_s1['area'] < 50], 
            pb_data_s1['rise_time'][pb_data_s1['area'] < 50], 
            s=5, alpha = 0.5, c = pb_data_s1['area_fraction_top'][pb_data_s1['area'] < 50],
            vmin=vmin, vmax=vmax, label = 'SOM S1',
            cmap=cm.get_cmap('Greens'))
plt.colorbar(m2, label = 'S1 AFT')
plt.plot(area_linspace, compute_s1_boundary(parm2, area_linspace),
         '--' ,color = 'black', label = 'Straxen Boundary')
#plt.scatter(kr_pb_array['area'][kr_pb_array['type'] == 1], kr_pb_array['rise_time'][kr_pb_array['type'] == 1], s=0.1, color = 'blue')
plt.xscale('log')
plt.yscale('log')
plt.xlabel('Area [PE]')
plt.ylabel('Rise Time [ns]')
legend_elements = [Line2D([0], [0], marker='o', color='green', label='SOM S1',markerfacecolor='green',linewidth=0, markersize=5),
                   Line2D([0], [0], marker='o', color='red', label='SOM S2',markerfacecolor='red',linewidth=0, markersize=5),
                   Line2D([0], [0], linestyle='--', color='black', label='Straxen Boundary',markerfacecolor='black',linewidth=2, markersize=5)]

plt.legend(handles=legend_elements, loc=0,prop={'size':12})
#plt.colorbar()
#plt.legend(markerscale=4)
plt.xlim(1,50)
plt.ylim(10,500)

In [ ]:
fig = plt.figure(figsize=(8,5))

vmin = 0
vmax = 1
m1 = plt.scatter(pb_data_s2['area'][pb_data_s2['area'] < 70], 
            pb_data_s2['rise_time'][pb_data_s2['area'] < 70], 
            s=5, alpha = 0.5, c= pb_data_s2['area_fraction_top'][pb_data_s2['area'] < 70],
            vmin=vmin, vmax=vmax, label = 'SOM S2', 
            cmap=cm.get_cmap('Greens'))
plt.colorbar(m1, label = 'S2 AFT')
m2 = plt.scatter(pb_data_s1['area'][pb_data_s1['area'] < 50], 
            pb_data_s1['rise_time'][pb_data_s1['area'] < 50], 
            s=5, alpha = 0.5, c = pb_data_s1['area_fraction_top'][pb_data_s1['area'] < 50],
            vmin=vmin, vmax=vmax, label = 'SOM S1',
            cmap=cm.get_cmap('Blues'))
plt.colorbar(m2, label = 'S1 AFT')
plt.plot(area_linspace, compute_s1_boundary(parm2, area_linspace),
         '--' ,color = 'black', label = 'Straxen Boundary')
#plt.scatter(kr_pb_array['area'][kr_pb_array['type'] == 1], kr_pb_array['rise_time'][kr_pb_array['type'] == 1], s=0.1, color = 'blue')
#plt.xscale('log')
#plt.yscale('log')
plt.xlabel('Area [PE]')
plt.ylabel('Rise Time [ns]')
legend_elements = [Line2D([0], [0], marker='o', color='blue', label='SOM S1',markerfacecolor='blue',linewidth=0, markersize=5),
                   Line2D([0], [0], marker='o', color='green', label='SOM S2',markerfacecolor='green',linewidth=0, markersize=5),
                   Line2D([0], [0], linestyle='--', color='black', label='Straxen Boundary',markerfacecolor='black',linewidth=2, markersize=5)]

plt.legend(handles=legend_elements, loc=0,prop={'size':12})
#plt.colorbar()
#plt.legend(markerscale=4)
plt.xlim(1,50)
plt.ylim(10,500)
plt.savefig('./figures/SOMcls_risetime_v_area_wstrax_bound.png')

In [ ]:
#pb_s1_data = pb_data[pb_data['type'] == 1]
pb_s1_data_high_aft_som = pb_data_s1[pb_data_s1['area_fraction_top'] > 0.8]

#pb_s2_data = pb_data[pb_data['type'] == 2]
pb_s2_data_low_aft_som = pb_data_s2[pb_data_s2['area_fraction_top'] < 0.2]

In [ ]:
fig = plt.figure(figsize=(6,5))

plt.scatter(pb_data_s1['area'], 
            pb_data_s1['rise_time'], 
            s=1, alpha = 0.5, color = 'blue', label = 'S1')
plt.scatter(pb_data_s2['area'], 
            pb_data_s2['rise_time'], 
            s=1, alpha = 0.5, color = 'green', label = 'S2')
plt.plot(area_linspace, compute_s1_boundary(parm2, area_linspace),
         '--' ,color = 'black', label = "Straxen Peaklet Boundary")
#plt.scatter(kr_pb_array['area'][kr_pb_array['type'] == 1], kr_pb_array['rise_time'][kr_pb_array['type'] == 1], s=0.1, color = 'blue')
plt.xscale('log')
plt.yscale('log')
plt.xlabel('Area [PE]')
plt.ylabel('Rise Time [ns]')
plt.xlim(1,50)
plt.ylim(10,500)
plt.legend(markerscale=8)
#plt.title("")

In [ ]:
fig = plt.figure(figsize=(6,5))

plt.scatter(pb_data_s1['area'], 
            pb_data_s1['rise_time'], 
            s=10, alpha = 0.5, color = 'blue', label = 'S1')
plt.scatter(pb_data_s2['area'], 
            pb_data_s2['rise_time'], 
            s=10, alpha = 0.5, color = 'green', label = 'S2')
plt.plot(area_linspace, compute_s1_boundary(parm2, area_linspace),
         '--' ,color = 'black', label = "Straxen Peaklet Boundary")
#plt.scatter(kr_pb_array['area'][kr_pb_array['type'] == 1], kr_pb_array['rise_time'][kr_pb_array['type'] == 1], s=0.1, color = 'blue')
#plt.xscale('log')
#plt.yscale('log')
plt.xlabel('Area [PE]')
plt.ylabel('Rise Time [ns]')
plt.xlim(5,20)
plt.ylim(50,200)
plt.legend(markerscale=8)
#plt.title("")

In [ ]:
fig = plt.figure(figsize=(7,5))

parm2 = np.array([50, 12, 80])
vmin = 0
vmax = 1
plt.scatter(pb_data_s1['area'], 
            pb_data_s1['rise_time'], 
            c = pb_data_s1['area_fraction_top'],
            s=10, alpha = 0.5, marker = '^',
            vmin=vmin, vmax=vmax, label = 'SOM S1',
            cmap=cm.get_cmap('viridis'))
plt.scatter(pb_data_s2['area'], 
            pb_data_s2['rise_time'], 
            c = pb_data_s2['area_fraction_top'],
            s=10, alpha = 0.5, vmin=vmin, vmax=vmax, 
            label = 'SOM S2', marker = 'v',
            cmap=cm.get_cmap('viridis'))
plt.scatter(pb_s1_data_high_aft_som['area'],
            pb_s1_data_high_aft_som['rise_time'], s = 10,
            marker = 'x', color = 'red', label = 'aft > 0.8, aft SOM S1')
plt.scatter(pb_s2_data_low_aft_som['area'],
            pb_s2_data_low_aft_som['rise_time'], s = 10,
            marker = 'x', color = 'black', label = 'aft < 0.2, aft SOM S2')
plt.plot(area_linspace, compute_s1_boundary(parm2, area_linspace),
         '--' ,color = 'black', label = 'Straxen Peaklet Boundary')
plt.colorbar(label = 'AFT')
#plt.scatter(kr_pb_array['area'][kr_pb_array['type'] == 1], kr_pb_array['rise_time'][kr_pb_array['type'] == 1], s=0.1, color = 'blue')
#plt.xscale('log')
#plt.yscale('log')
plt.xlabel('Area [PE]')
plt.ylabel('Rise Time [ns]')
plt.legend(markerscale=3, loc = 1)
plt.xlim(5,20)
plt.ylim(50,200)

In [ ]:
pb_s1_data_st = pb_data[pb_data['type'] == 1]
pb_s1_data_high_aft_st = pb_s1_data_st[pb_s1_data_st['area_fraction_top'] > 0.8]

pb_s2_data_st = pb_data[pb_data['type'] == 2]
pb_s2_data_low_aft_st = pb_s2_data_st[pb_s2_data_st['area_fraction_top'] < 0.2]

In [ ]:
pb_s1_data_high_aft_st['area_fraction_top']

In [ ]:
fig = plt.figure(figsize=(6,5))

plt.scatter(pb_s1_data_st['area'], 
            pb_s1_data_st['rise_time'], 
            s=1, alpha = 0.5, color = 'blue', label = 'S1')
plt.scatter(pb_s2_data_st['area'], 
            pb_s2_data_st['rise_time'], 
            s=1, alpha = 0.5, color = 'green', label = 'S2')
plt.plot(area_linspace, compute_s1_boundary(parm2, area_linspace),
         '--' ,color = 'black', label = "Straxen Peaklet Boundary")
#plt.scatter(kr_pb_array['area'][kr_pb_array['type'] == 1], kr_pb_array['rise_time'][kr_pb_array['type'] == 1], s=0.1, color = 'blue')
plt.xscale('log')
plt.yscale('log')
plt.xlabel('Area [PE]')
plt.ylabel('Rise Time [ns]')
plt.xlim(1,50)
plt.ylim(10,500)
plt.legend(markerscale=8)
#plt.title("")

In [ ]:
upper = 0.6
lower = 0.4
#pb_s1_data = pb_data[pb_data['type'] == 1]
pb_s1_data_high_aft_som = pb_data_s1[pb_data_s1['area_fraction_top'] > upper]

#pb_s2_data = pb_data[pb_data['type'] == 2]
pb_s2_data_low_aft_som = pb_data_s2[pb_data_s2['area_fraction_top'] < lower]

In [ ]:
pb_s1_data_st = pb_data[pb_data['type'] == 1]
pb_s1_data_high_aft_st = pb_s1_data_st[pb_s1_data_st['area_fraction_top'] > upper]

pb_s2_data_st = pb_data[pb_data['type'] == 2]
pb_s2_data_low_aft_st = pb_s2_data_st[pb_s2_data_st['area_fraction_top'] < lower]

In [ ]:
def num_of_events_per_aft(data, split, hl):
    """
    data: peak basics
    split: nplinespace with what you need
    hl: 0 for data less than a certain aft
        1 for data more than a certain aft
    """
    #split = np.linspace(min_aft, max_aft, num_of_splits)
    results = np.zeros(len(split))
    for i in np.arange(len(split)):
        if hl == 0:
            results[i] = len(data[data['area_fraction_top'] < split[i]])
        if hl == 1:
            results[i] = len(data[data['area_fraction_top'] > split[i]])
            
    return results

In [ ]:
high_aft = np.linspace(0.6,1,25)
low_aft = np.linspace(0,0.4,25)

In [ ]:
fig = plt.figure(figsize=(5.5,4))
plt.scatter(high_aft ,num_of_events_per_aft(pb_s1_data_st, high_aft, 1), marker = '.',
            label = 'Straxen # of S1s high AFT', color = 'orange')
plt.scatter(high_aft ,num_of_events_per_aft(pb_data_s1, high_aft, 1), marker = 'x',
           label = 'SOM # of S1s high AFT', color = 'blue')
plt.legend()
plt.xlabel("AFT")
plt.ylabel('Number of events')
plt.savefig('./figures/high_aft_S1.png', dpi = 1200)

In [ ]:
fig = plt.figure(figsize=(5.5,4))
plt.scatter(low_aft ,num_of_events_per_aft(pb_s2_data_st, low_aft, 0), marker = '.',
           label = 'Straxen # of S2s low AFT', color = 'orange')
plt.scatter(low_aft ,num_of_events_per_aft(pb_data_s2, low_aft, 0), marker = 'x',
           label = 'SOM # of S2s low AFT', color = 'blue')
plt.gca().invert_xaxis()
plt.legend()
plt.xlabel("AFT")
plt.ylabel('Number of events')
plt.savefig('./figures/low_aft_S2.png', dpi = 1200)

In [ ]:
np.linspace(0.6,1,25)

In [ ]:
len(pb_s1_data_high_aft_som)

In [ ]:
len(pb_s1_data_high_aft_st)

In [ ]:
len(pb_s2_data_low_aft_som)

In [ ]:
len(pb_s2_data_low_aft_st)

In [ ]:
fig = plt.figure(figsize=(7,5))

parm2 = np.array([50, 12, 80])
vmin = 0
vmax = 1
plt.scatter(pb_data_s1['area'], 
            pb_data_s1['rise_time'], 
            c = pb_data_s1['area_fraction_top'],
            s=10, alpha = 0.5, marker = '^',
            vmin=vmin, vmax=vmax, label = 'SOM S1',
            cmap=cm.get_cmap('viridis'))
plt.scatter(pb_data_s2['area'], 
            pb_data_s2['rise_time'], 
            c = pb_data_s2['area_fraction_top'],
            s=10, alpha = 0.5, vmin=vmin, vmax=vmax, 
            label = 'SOM S2', marker = 'v',
            cmap=cm.get_cmap('viridis'))
plt.scatter(pb_s1_data_high_aft_som['area'],
            pb_s1_data_high_aft_som['rise_time'], s = 10,
            marker = 'x', color = 'red', label = 'aft > 0.8, aft SOM S1')
plt.scatter(pb_s2_data_low_aft_som['area'],
            pb_s2_data_low_aft_som['rise_time'], s = 10,
            marker = 'x', color = 'black', label = 'aft < 0.2, aft SOM S2')
plt.plot(area_linspace, compute_s1_boundary(parm2, area_linspace),
         '--' ,color = 'black', label = 'Straxen Peaklet Boundary')
plt.colorbar(label = 'AFT')
#plt.scatter(kr_pb_array['area'][kr_pb_array['type'] == 1], kr_pb_array['rise_time'][kr_pb_array['type'] == 1], s=0.1, color = 'blue')
#plt.xscale('log')
#plt.yscale('log')
plt.xlabel('Area [PE]')
plt.ylabel('Rise Time [ns]')
plt.legend(markerscale=3, loc = 1)
plt.xlim(1,50)
plt.ylim(70,125)
plt.savefig('./figures/Risetime_v_area_SOM_error.png', dpi = 1200)

In [ ]:
area_linspace =  np.linspace(1, 50, 100)

In [ ]:
parm = np.array([50, 80, 12])
#boundary_line = parm[0]*np.exp(-area/tau) + const

In [ ]:
fig = plt.figure(figsize=(10,7))

vmin = 0
vmax = 1
plt.scatter(pb_data_s2['area'][pb_data_s2['area'] < 70], 
            pb_data_s2['rise_time'][pb_data_s2['area'] < 70], 
            s=0.5, alpha = 0.5, c= pb_data_s2['area_fraction_top'][pb_data_s2['area'] < 70],
            vmin=vmin, vmax=vmax,
            cmap=cm.get_cmap('jet'))
plt.scatter(pb_data_s1['area'][pb_data_s1['area'] < 50], 
            pb_data_s1['rise_time'][pb_data_s1['area'] < 50], 
            s=0.5, alpha = 0.5, c = pb_data_s1['area_fraction_top'][pb_data_s1['area'] < 50],
            vmin=vmin, vmax=vmax,
            cmap=cm.get_cmap('jet'))
plt.plot(area_linspace, compute_s1_boundary(parm, area_linspace))
#plt.scatter(kr_pb_array['area'][kr_pb_array['type'] == 1], kr_pb_array['rise_time'][kr_pb_array['type'] == 1], s=0.1, color = 'blue')
plt.xscale('log')
plt.yscale('log')
plt.xlabel('Area [PE]')
plt.ylabel('Rise Time [ns]')
plt.colorbar()
plt.xlim(1,50)
plt.ylim(10,500)

In [ ]:
aft_bound_parm = np.array([-1, 2.6, 0.6, 100])

In [ ]:
'''strax.Option('s1_risetime_aft_parameters', default=(-1, 2.6), type=(list, tuple),
                 help=("Slope and offset in exponential of emperical boundary in the rise time-AFT "
                      "plot. Specified as (slope, offset)")),
strax.Option('s1_flatten_threshold_aft', default=(0.6, 100), type=(tuple, list),
                 help=("Threshold for AFT, above which we use a flatted boundary for rise time" 
                       "Specified values: (AFT boundary, constant rise time).")),'''

In [ ]:
def compute_s1_boundary(parm, area):
    boundary_line = parm[0]*np.exp(-area/parm[1]) + parm[2]
    
    return boundary_line

In [ ]:
v = compute_s1_boundary(parm, area_linspace)

In [ ]:
v

In [ ]:
def upper_rise_time_aft_boundary(aft, slope, offset, aft_boundary, flat_threshold):
        """
        Function which computes the upper rise time boundary as a function
        of area fraction top.
        """
        res = 10**(slope * aft + offset)
        res[aft >= aft_boundary] = flat_threshold
        return res

In [ ]:
parm = np.array([50, 80, 12])
boundary_line = parm[0]*np.exp(-area/tau) + const

In [ ]:
fig = plt.figure(figsize=(6,5))

vmin = 0
vmax = 1
plt.scatter(pb_data_s2['area'][pb_data_s2['area'] < 70], 
            pb_data_s2['rise_time'][pb_data_s2['area'] < 70], 
            s=1, alpha = 0.5, c= pb_data_s2['area_fraction_top'][pb_data_s2['area'] < 70],
            vmin=vmin, vmax=vmax,
            cmap=cm.get_cmap('jet'))
plt.scatter(pb_data_s1['area'][pb_data_s1['area'] < 50], 
            pb_data_s1['rise_time'][pb_data_s1['area'] < 50], 
            s=0.5, alpha = 0.5, c = pb_data_s1['area_fraction_top'][pb_data_s1['area'] < 50],
            vmin=vmin, vmax=vmax,
            cmap=cm.get_cmap('jet'))
#plt.scatter(kr_pb_array['area'][kr_pb_array['type'] == 1], kr_pb_array['rise_time'][kr_pb_array['type'] == 1], s=0.1, color = 'blue')
#plt.xscale('log')
#plt.yscale('log')
plt.xlabel('Area [PE]')
plt.ylabel('Rise Time [ns]')
plt.colorbar()
plt.xlim(1,50)
plt.ylim(10,300)

In [ ]:
pb_data_s2.dtype

In [ ]:
fig = plt.figure(figsize=(8,5))

vmin = 0
vmax = 200
n1 = plt.scatter(pb_data_s2['area_fraction_top'][pb_data_s2['area'] < 70], 
            pb_data_s2['rise_time'][pb_data_s2['area'] < 70], 
            s=5, alpha = 0.5, c= pb_data_s2['range_50p_area'][pb_data_s2['area'] < 70],
            vmin=vmin, vmax=vmax,
            cmap=cm.get_cmap('Greens'))
plt.colorbar(n1, label = 'S2 50% width [ns]')
n2 = plt.scatter(pb_data_s1['area_fraction_top'][pb_data_s1['area'] < 50], 
            pb_data_s1['rise_time'][pb_data_s1['area'] < 50], 
            s=5, alpha = 0.5, c = pb_data_s1['range_50p_area'][pb_data_s1['area'] < 50],
            vmin=vmin, vmax=vmax,
            cmap=cm.get_cmap('Blues'))
plt.colorbar(n2, label = 'S1 50% width [ns]')
plt.plot(aft_line, upper_rise_time_aft_boundary(aft_line, aft_bound_parm[0],
                                               aft_bound_parm[1],aft_bound_parm[2],
                                               aft_bound_parm[3]),'--' ,color = 'black', label = "Straxen AFT boundary")
#plt.scatter(kr_pb_array['area'][kr_pb_array['type'] == 1], kr_pb_array['rise_time'][kr_pb_array['type'] == 1], s=0.1, color = 'blue')
#plt.xscale('log')
legend_elements = [Line2D([0], [0], marker='o', color='blue', label='SOM S1',markerfacecolor='blue',linewidth=0, markersize=5),
                   Line2D([0], [0], marker='o', color='green', label='SOM S2',markerfacecolor='green',linewidth=0, markersize=5),
                   Line2D([0], [0], linestyle='--', color='black', label='Straxen Boundary',markerfacecolor='black',linewidth=2, markersize=5)]

plt.legend(handles=legend_elements, loc=0,prop={'size':12})
plt.yscale('log')
plt.xlabel('Area Fraction Top')
plt.ylabel('Rise Time [ns]')
#plt.colorbar(label = '50% width [ns]')
plt.xlim(0,1)
plt.ylim(10,1000)
plt.savefig('./figures/SOMcls_risetime_v_AFT_wstrax_bound.png')

In [ ]:
fig = plt.figure(figsize=(6,5))

vmin = 0
vmax = 200
plt.scatter(pb_data_s2['area_fraction_top'][pb_data_s2['area'] < 70], 
            pb_data_s2['rise_time'][pb_data_s2['area'] < 70], 
            s=0.5, alpha = 0.5, c= pb_data_s2['range_50p_area'][pb_data_s2['area'] < 70],
            vmin=vmin, vmax=vmax,
            cmap=cm.get_cmap('jet'))
plt.scatter(pb_data_s1['area_fraction_top'][pb_data_s1['area'] < 50], 
            pb_data_s1['rise_time'][pb_data_s1['area'] < 50], 
            s=0.5, alpha = 0.5, c = pb_data_s1['range_50p_area'][pb_data_s1['area'] < 50],
            vmin=vmin, vmax=vmax,
            cmap=cm.get_cmap('jet'))
plt.plot(aft_line, upper_rise_time_aft_boundary(aft_line, aft_bound_parm[0],
                                               aft_bound_parm[1],aft_bound_parm[2],
                                               aft_bound_parm[3]),'--' ,color = 'black', label = "Straxen AFT boundary")
#plt.scatter(kr_pb_array['area'][kr_pb_array['type'] == 1], kr_pb_array['rise_time'][kr_pb_array['type'] == 1], s=0.1, color = 'blue')
#plt.xscale('log')
plt.yscale('log')
plt.xlabel('Area [PE]')
plt.ylabel('Rise Time [ns]')
plt.colorbar(label = '50% width [ns]')
plt.xlim(0,1)
plt.ylim(10,1000)

In [ ]:
fig = plt.figure(figsize=(10,7))

vmin = 0
vmax = 200
plt.scatter(pb_data_s2['rise_time'][pb_data_s2['area'] < 70], 
            pb_data_s2['rise_time'][pb_data_s2['area'] < 70], 
            s=0.5, alpha = 0.5, color = 'blue')
plt.scatter(pb_data_s1['rise_time'][pb_data_s1['area'] < 50], 
            pb_data_s1['rise_time'][pb_data_s1['area'] < 50], 
            s=0.5, alpha = 0.5, color = 'green')
#plt.scatter(kr_pb_array['area'][kr_pb_array['type'] == 1], kr_pb_array['rise_time'][kr_pb_array['type'] == 1], s=0.1, color = 'blue')
#plt.xscale('log')
#plt.yscale('log')
plt.xlabel('Area [PE]')
plt.ylabel('Rise Time [ns]')
plt.xlim(0,1)
plt.ylim(10,1000)

In [ ]:
fig = plt.figure(figsize=(10,7))

plt.scatter(pb_data_SOM2cls['area'][pb_data_SOM2cls['type'] == 1], 
            pb_data_SOM2cls['rise_time'][pb_data_SOM2cls['type'] == 1], 
            s=0.5, alpha = 0.5, color = 'blue')
plt.scatter(pb_data_SOM2cls['area'][pb_data_SOM2cls['type'] == 2], 
            pb_data_SOM2cls['rise_time'][pb_data_SOM2cls['type'] == 2], 
            s=0.5, alpha = 0.5, color = 'green')
#plt.scatter(kr_pb_array['area'][kr_pb_array['type'] == 1], kr_pb_array['rise_time'][kr_pb_array['type'] == 1], s=0.1, color = 'blue')
#plt.xscale('log')
#plt.yscale('log')
plt.xlabel('Area [PE]')
plt.ylabel('Rise Time [ns]')
plt.xlim(1,1000000)
plt.ylim(1,100000)

In [ ]:
fig = plt.figure(figsize=(10,7))

plt.scatter(data_pb_cuts_2clsSOM['area'][data_pb_cuts_2clsSOM['type'] == 1], 
            data_pb_cuts_2clsSOM['rise_time'][data_pb_cuts_2clsSOM['type'] == 1], 
            s=0.5, alpha = 0.5, color = 'blue')
plt.scatter(data_pb_cuts_2clsSOM['area'][data_pb_cuts_2clsSOM['type'] == 2], 
            data_pb_cuts_2clsSOM['rise_time'][data_pb_cuts_2clsSOM['type'] == 2], 
            s=0.5, alpha = 0.5, color = 'green')
#plt.scatter(kr_pb_array['area'][kr_pb_array['type'] == 1], kr_pb_array['rise_time'][kr_pb_array['type'] == 1], s=0.1, color = 'blue')
plt.xscale('log')
plt.yscale('log')
plt.xlabel('Area [PE]')
plt.ylabel('Rise Time [ns]')
plt.xlim(1,1000000)
plt.ylim(1,100000)